<a href="https://colab.research.google.com/github/subhamulal/imageclassification1/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [9]:
# Step 1: Install Git if needed
!apt-get install git

# Step 2: Configure Git (Use your GitHub email & name)
!git config --global user.email "your-email@example.com"
!git config --global user.name "YourGitHubUsername"

# Step 3: Clone your GitHub repository (Run only once)
!git clone https://github.com/YourGitHubUsername/YourRepositoryName.git
%cd YourRepositoryName

# Step 4: Copy the Colab notebook to the repo folder
!cp /content/YourNotebook.ipynb /content/YourRepositoryName/

# Step 5: Add, commit, and push changes
!git add .
!git commit -m "Updated notebook with new code"
!git push origin main  # Change 'main' to your branch if different


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
Cloning into 'YourRepositoryName'...
fatal: could not read Username for 'https://github.com': No such device or address
[Errno 2] No such file or directory: 'YourRepositoryName'
/content
cp: cannot stat '/content/YourNotebook.ipynb': No such file or directory
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
